In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
df_balance= pd.read_csv("./data/us-balance-quarterly.csv", sep=";")
df_cashflow = pd.read_csv("./data/us-cashflow-quarterly.csv", sep=";")
df_income = pd.read_csv("./data/us-income-quarterly.csv", sep=";")

In [4]:
df_prices_copy = pd.read_csv("./data/us-shareprices-daily.csv", sep=";")
df_prices = df_prices_copy.copy()
df_prices.loc['Date'] = pd.to_datetime(df_prices['Date'], errors = 'coerce')

In [5]:
from pandas.tseries.offsets import DateOffset

def get_price_on_or_after(df_prices, simfinid, target_date):
    prices = df_prices[df_prices['SimFinId'] == simfinid]
    return prices[prices['Date'] >= target_date].sort_values('Date').head(1)

def get_price_3m_later(df_prices, simfinid, publish_date):
    future_date = publish_date + DateOffset(months=3)
    return get_price_on_or_after(df_prices, simfinid, future_date)

In [6]:
df = pd.merge(df_balance, df_cashflow, on='SimFinId', how= 'outer')
df = pd.merge(df, df_income, on ='SimFinId', how='outer')

In [7]:
df.head(5)

,Ticker_x,SimFinId,Currency_x,Fiscal Year_x,Fiscal Period_x,Report Date_x,Publish Date_x,Restated Date_x,Shares (Basic)_x,Shares (Diluted)_x,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
0,GOOG,18,USD,2019.0,Q2,2019-06-30,2019-07-26,2019-07-26,1.387938e+10,1.492958e+10,...,2.967000e+09,627056859.0,1.214700e+10,0.0,1.214700e+10,-2.200000e+09,9.947000e+09,NaN,9.947000e+09,9.947000e+09
1,GOOG,18,USD,2019.0,Q2,2019-06-30,2019-07-26,2019-07-26,1.387938e+10,1.492958e+10,...,-5.490000e+08,608046614.0,8.628000e+09,0.0,8.628000e+09,-1.560000e+09,7.068000e+09,NaN,7.068000e+09,7.068000e+09
2,GOOG,18,USD,2019.0,Q2,2019-06-30,2019-07-26,2019-07-26,1.387938e+10,1.492958e+10,...,1.438000e+09,603208347.0,1.070400e+10,0.0,1.070400e+10,-3.300000e+07,1.067100e+10,NaN,1.067100e+10,1.067100e+10
3,GOOG,18,USD,2019.0,Q2,2019-06-30,2019-07-26,2019-07-26,1.387938e+10,1.492958e+10,...,-2.200000e+08,563792119.0,7.757000e+09,NaN,7.757000e+09,-9.210000e+08,6.836000e+09,NaN,6.836000e+09,6.836000e+09
4,GOOG,18,USD,2019.0,Q2,2019-06-30,2019-07-26,2019-07-26,1.387938e+10,1.492958e+10,...,1.894000e+09,420441603.0,8.277000e+09,NaN,8.277000e+09,-1.318000e+09,6.959000e+09,NaN,6.959000e+09,6.959000e+09


In [8]:
df_income['Publish Date'] = pd.to_datetime(df_income['Publish Date'], errors = 'coerce')

In [ ]:
yield_data = []

for idx, row in df_income.iterrows():
    simfinid = row['SimFinId']
    publish_date = row['Publish Date']
    
    try:
        price_t = get_price_on_or_after(df_prices, simfinid, publish_date)
        price_t3m = get_price_3m_later(df_prices, simfinid, publish_date)

        if not price_t.empty and not price_t3m.empty:
            p0 = price_t['Adj. Close'].values[0]
            p1 = price_t3m['Adj. Close'].values[0]
            yield_3m = (p1 - p0) / p0
        else:
            yield_3m = None

    except Exception as e:
        # if idx % 1000 == 0:
        #     print(f"Error at {idx}: {e}")
        yield_3m = None
    
    yield_data.append(yield_3m)

df['Yield_3M'] = yield_data

In [ ]:
import yfinance as yf

In [ ]:
# importo datos de precios de yfinance para luego matchear con df original
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

all_data = []
failed_tickers = []

for ticker in tickers:
    try:
        df_prices = yf.download(ticker, start="2019-01-01", end="2024-01-01", progress=False)
        
        if df_prices.empty:
            failed_tickers.append(ticker)
            continue
        
        df_prices = df_prices.reset_index()
        df_prices['Ticker'] = ticker
        all_data.append(df_prices)
        
    except Exception as e:
        failed_tickers.append(ticker)

# Combine all successful downloads
df_all = pd.concat(all_data, ignore_index=True)

print(f"\n✅ Download complete: {len(all_data)} tickers succeeded, {len(failed_tickers)} failed.")
successful_tickers = [df_prices['Ticker'].iloc[0] for df_prices in all_data if not df_prices.empty]
pd.Series(successful_tickers).to_csv("./data/successful_tickers.csv", index=False)